Input Data:
- '../Clean_Data/Weather_Data_w_Veg_Filter_FOD_FPA_Fire_12KM/wind_speed_filtered.parquet' (from code 02 04)
- '../Clean_Data/Fire_Data/calfire_fod_fpa_2001_2020_dedup_map_weather_grid.parquet' (from code 00)

Output Data:
- '../Clean_Data/Fire_Data/calfire_fod_fpa_2001_2020_fire_label_w_size.parquet'


In [68]:
import pandas as pd
import geopandas as gpd
import xarray as xr
import pyproj
from tqdm import tqdm
import numpy as np

In [132]:
import re

In [69]:
import os
import matplotlib.pyplot as plt
#import cartopy.crs as ccrs
#import cartopy.feature as cfeature

In [70]:
# check python version and all packages version
def check_python_version():
    import sys
    print("Python version")
    print (sys.version)
    print("Pandas version")
    print(pd.__version__)
    print("Geopandas version")
    print(gpd.__version__)
    print("Xarray version")
    print(xr.__version__)
    print("Pyproj version")
    print(pyproj.__version__)

check_python_version()

Python version
3.11.9 | packaged by Anaconda, Inc. | (main, Apr 19 2024, 16:40:41) [MSC v.1916 64 bit (AMD64)]
Pandas version
2.2.2
Geopandas version
0.14.2
Xarray version
2023.6.0
Pyproj version
3.6.1


In [71]:
weather_data = pd.read_parquet('../Clean_Data/Weather_Data_w_Veg_Filter_FOD_FPA_Fire_12KM/wind_speed_filtered.parquet')

In [72]:
weather_data.shape # this weather data is after inner join with vegetation data, within 12km of fire event

(147252000, 4)

In [73]:
# only keep columns (lon, lat, and day)
weather_data = weather_data[['lon', 'lat', 'day']]

In [7]:
# drop duplicates
weather_data = weather_data.drop_duplicates()

In [74]:
weather_data.shape

(147252000, 3)

In [75]:
# check if there is any missing value
weather_data.isnull().sum()

lon    0
lat    0
day    0
dtype: int64

In [133]:
fire_data = pd.read_parquet('../Clean_Data/Fire_Data/calfire_fod_fpa_2001_2020_dedup_map_weather_grid.parquet')

In [134]:
fire_data['NWCG_CAUSE_CLASSIFICATION'] = fire_data['NWCG_CAUSE_CLASSIFICATION'].apply(str)

In [86]:
fire_data.head()

,LONGITUDE,LATITUDE,DISCOVERY_DATE,CONT_DATE,max_FIRE_SIZE,min_FIRE_SIZE,FOD_ID_cnt,NWCG_CAUSE_CLASSIFICATION,diff_FIRE_SIZE,NEAREST_LAT,NEAREST_LON,DISTANCE_KM
0,-123.943617,40.476117,10/28/2015,10/28/2015,0.01,0.01,18,[Human],0.0,40.483333,-123.933333,1.184269
1,-120.850714,38.641597,7/16/2006,7/17/2006,13.80,0.10,14,[Human],13.7,38.650000,-120.850000,0.934875
2,-121.607969,39.806769,11/28/2003,11/28/2003,0.10,0.10,11,[Human],0.0,39.816667,-121.600000,1.293568
3,-123.163889,39.678056,10/28/2005,10/28/2005,0.10,0.10,11,[Human],0.0,39.691667,-123.183333,2.250813
4,-120.710000,38.293056,5/17/2007,missing date,7.00,1.00,9,[Human],6.0,38.275000,-120.725000,2.395657


In [135]:
# if CONT_DATE = "missing date", assign it to NA
fire_data['CONT_DATE'] = fire_data['CONT_DATE'].replace('missing date', np.nan)

In [136]:
# Convert date columns to datetime
fire_data['DISCOVERY_DATE'] = pd.to_datetime(fire_data['DISCOVERY_DATE'])
fire_data['CONT_DATE'] = pd.to_datetime(fire_data['CONT_DATE'])

In [137]:
fire_data['fire_days'] = (fire_data['CONT_DATE'] - fire_data['DISCOVERY_DATE']).dt.days

In [14]:
fire_data['fire_days'].quantile([0.75, 0.9, 0.95, 0.99])
# providing evidence that most of the fire events are short-lived, 99% have less than 16 days

0.75     0.0
0.90     1.0
0.95     2.0
0.99    16.0
Name: fire_days, dtype: float64

In [138]:
# for those fire with missing CONT_DATE, assign CONT_DATE to DISCOVERY_DATE + 16 days
fire_data['CONT_DATE'] = fire_data['CONT_DATE'].fillna(fire_data['DISCOVERY_DATE'] + pd.DateOffset(days=16))

In [17]:
fire_data.head()

,LONGITUDE,LATITUDE,DISCOVERY_DATE,CONT_DATE,max_FIRE_SIZE,min_FIRE_SIZE,FOD_ID_cnt,NWCG_CAUSE_CLASSIFICATION,diff_FIRE_SIZE,NEAREST_LAT,NEAREST_LON,DISTANCE_KM,fire_days
0,-123.943617,40.476117,2015-10-28,2015-10-28,0.01,0.01,18,[Human],0.0,40.483333,-123.933333,1.184269,0.0
1,-120.850714,38.641597,2006-07-16,2006-07-17,13.80,0.10,14,[Human],13.7,38.650000,-120.850000,0.934875,1.0
2,-121.607969,39.806769,2003-11-28,2003-11-28,0.10,0.10,11,[Human],0.0,39.816667,-121.600000,1.293568,0.0
3,-123.163889,39.678056,2005-10-28,2005-10-28,0.10,0.10,11,[Human],0.0,39.691667,-123.183333,2.250813,0.0
4,-120.710000,38.293056,2007-05-17,2007-06-02,7.00,1.00,9,[Human],6.0,38.275000,-120.725000,2.395657,NaN


In [19]:
# check if there is any missing value
fire_data.isnull().sum()

LONGITUDE                        0
LATITUDE                         0
DISCOVERY_DATE                   0
CONT_DATE                        0
max_FIRE_SIZE                    0
min_FIRE_SIZE                    0
FOD_ID_cnt                       0
NWCG_CAUSE_CLASSIFICATION        0
diff_FIRE_SIZE                   0
NEAREST_LAT                      0
NEAREST_LON                      0
DISTANCE_KM                      0
fire_days                    75735
dtype: int64

In [139]:
fire_data['NWCG_CAUSE_CLASSIFICATION'].value_counts()

NWCG_CAUSE_CLASSIFICATION
['Human']                                                118638
['Missing data/not specified/undetermined']               37841
['Natural']                                               18668
['Human' 'Missing data/not specified/undetermined']          95
['Human' 'Natural']                                          14
['Missing data/not specified/undetermined' 'Natural']         1
Name: count, dtype: int64

In [ ]:
columns_to_show = ['NEAREST_LON','NEAREST_LAT','DISCOVERY_DATE','CONT_DATE','NWCG_CAUSE_CLASSIFICATION']
fire_data[columns_to_show].duplicated().sum()

4066

In [82]:
columns_to_show = ['NEAREST_LON','NEAREST_LAT','DISCOVERY_DATE','CONT_DATE']
fire_data[columns_to_show].duplicated().sum()

4702

this means some fire events which happen to be in the same dates and are pretty close so being matched to the same weather grid.

In [140]:
# Function to join unique values
def join_unique(values):
    unique_values = set(values)
    return ', '.join(unique_values)

In [141]:
print(f"before deduplication, the shape of fire data is {fire_data.shape}")
column_to_group = ['NEAREST_LON','NEAREST_LAT','DISCOVERY_DATE','CONT_DATE']
# group by columns_to_group, paste NWCG_CAUSE_CLASSIFICATION together
fire_data = fire_data.groupby(column_to_group)['NWCG_CAUSE_CLASSIFICATION'].apply(join_unique).reset_index()
print(f"after deduplication, the shape of fire data is {fire_data.shape}")

before deduplication, the shape of fire data is (175257, 13)
after deduplication, the shape of fire data is (170555, 5)


In [142]:
fire_data['NWCG_CAUSE_CLASSIFICATION'] = fire_data['NWCG_CAUSE_CLASSIFICATION'].str.replace('[', '').str.replace(']', '')

In [143]:
# Function to add commas between single-quoted strings if they are missing and deduplicate
def format_and_deduplicate(text):
    # Add commas between single-quoted strings if they are missing
    text = re.sub(r"(?<=')\s+(?=')", ", ", text)
    # Split the string into individual components
    components = re.findall(r"'[^']+'", text)
    # Remove duplicates by converting the list to a set, then back to a list
    unique_components = list(set(components))
    # Join the components back into a single string with proper formatting
    return ', '.join(unique_components)

# Apply the function to the NWCG_CAUSE_CLASSIFICATION column
fire_data['NWCG_CAUSE_CLASSIFICATION'] = fire_data['NWCG_CAUSE_CLASSIFICATION'].apply(format_and_deduplicate)

In [144]:
fire_data['NWCG_CAUSE_CLASSIFICATION'].value_counts()

NWCG_CAUSE_CLASSIFICATION
'Human'                                                 115416
'Missing data/not specified/undetermined'                36699
'Natural'                                                17703
'Missing data/not specified/undetermined', 'Human'         653
'Natural', 'Human'                                          79
'Missing data/not specified/undetermined', 'Natural'         5
Name: count, dtype: int64

In [145]:
fire_data.head()

,NEAREST_LON,NEAREST_LAT,DISCOVERY_DATE,CONT_DATE,NWCG_CAUSE_CLASSIFICATION
0,-124.391667,40.400000,2007-07-14,2007-07-14,'Human'
1,-124.391667,40.441667,2001-07-17,2001-07-17,'Human'
2,-124.391667,40.441667,2005-09-24,2005-09-24,'Human'
3,-124.391667,40.441667,2012-10-20,2012-11-05,'Human'
4,-124.391667,40.441667,2014-07-30,2014-07-30,'Human'


In [146]:
# check if there is rows that have CONT_DATE < DISCOVERY_DATE
(fire_data['CONT_DATE'] < fire_data['DISCOVERY_DATE']).sum()

0

For each location, for days
- alarm date -4, alarm date -1: remove
- alarm date: label 1
- alarm date - cont date: remove

In [147]:
fire_data = fire_data.rename(columns={'NEAREST_LON': 'lon', 'NEAREST_LAT': 'lat'})

In [148]:
# check if there is any duplicate
fire_data.duplicated().sum()

0

In [151]:
fire_data.dtypes

lon                                 float64
lat                                 float64
DISCOVERY_DATE               datetime64[ns]
CONT_DATE                    datetime64[ns]
NWCG_CAUSE_CLASSIFICATION            object
dtype: object

In [165]:
fire_data.shape

(170555, 5)

In [189]:
# Function to join unique values with proper comma separation
def join_unique(values):
    unique_values = set(values)
    joined_string = ', '.join(unique_values)
    # Split the string into individual components
    components = re.findall(r"'[^']+'", joined_string)
    unique_components = list(set(components))
    # order the unique components
    unique_components.sort()
    return ', '.join(unique_components)

# Group by the specified columns and aggregate
fire_cause_summary = fire_data.groupby(['lon', 'lat', 'DISCOVERY_DATE']).agg(
    NWCG_CAUSE_CLASSIFICATION=('NWCG_CAUSE_CLASSIFICATION', join_unique)
).reset_index()

In [190]:
# rename DISCOVERY_DATE to date
fire_cause_summary = fire_cause_summary.rename(columns={'DISCOVERY_DATE': 'day'})

In [191]:
fire_cause_summary['NWCG_CAUSE_CLASSIFICATION'].value_counts()

NWCG_CAUSE_CLASSIFICATION
'Human'                                                 114593
'Missing data/not specified/undetermined'                36453
'Natural'                                                17129
'Human', 'Missing data/not specified/undetermined'         844
'Human', 'Natural'                                         126
'Missing data/not specified/undetermined', 'Natural'         6
Name: count, dtype: int64

In [176]:
fire_cause_summary.shape

(169151, 4)

In [164]:
# Function to expand each row
def expand_dates(row):
    start_date = row['DISCOVERY_DATE'] - pd.Timedelta(days=4)
    end_date = row['CONT_DATE']
    date_range = pd.date_range(start=start_date, end=end_date)
    return pd.DataFrame({
        'lon': row['lon'],
        'lat': row['lat'],
        'DATE': date_range,
        'IS_DISCOVERY_DATE': date_range == row['DISCOVERY_DATE']
    })

# Apply the function to each row and concatenate the results
expanded_fire_data = pd.concat(fire_data.apply(expand_dates, axis=1).to_list(), ignore_index=True)

In [177]:
# check row 1: row 6
expanded_fire_data.iloc[0:6]

,lon,lat,DATE,IS_DISCOVERY_DATE
0,-124.391667,40.400000,2007-07-10,False
1,-124.391667,40.400000,2007-07-11,False
2,-124.391667,40.400000,2007-07-12,False
3,-124.391667,40.400000,2007-07-13,False
4,-124.391667,40.400000,2007-07-14,True
5,-124.391667,40.441667,2001-07-13,False


In [178]:
expanded_fire_data.shape

(2112512, 4)

In [179]:
# Group by the specified columns and aggregate
expanded_fire_data = expanded_fire_data.groupby(['lon', 'lat', 'DATE']).agg(
    IS_DISCOVERY_DATE=('IS_DISCOVERY_DATE', 'any')
).reset_index()

In [180]:
# check if there is any missing value
expanded_fire_data.isnull().sum()

lon                  0
lat                  0
DATE                 0
IS_DISCOVERY_DATE    0
dtype: int64

In [181]:
expanded_fire_data.shape

(1829209, 4)

In [182]:
# check if there is any duplicate  
expanded_fire_data.duplicated().sum()

0

In [194]:
expanded_fire_data = expanded_fire_data.rename(columns={'DATE': 'day'})

In [195]:
# freq table of IS_ALARM_DATE
expanded_fire_data['IS_DISCOVERY_DATE'].value_counts()

IS_DISCOVERY_DATE
False    1660058
True      169151
Name: count, dtype: int64

In [196]:
# merge fire_cause_summary and expanded_fire_data
expanded_fire_data = expanded_fire_data.merge(fire_cause_summary, on=['lon', 'lat', 'day'], how='left')

In [198]:
expanded_fire_data.head()

,lon,lat,day,IS_DISCOVERY_DATE,NWCG_CAUSE_CLASSIFICATION
0,-124.391667,40.4,2007-07-10,False,NaN
1,-124.391667,40.4,2007-07-11,False,NaN
2,-124.391667,40.4,2007-07-12,False,NaN
3,-124.391667,40.4,2007-07-13,False,NaN
4,-124.391667,40.4,2007-07-14,True,'Human'


In [199]:
# left join weather_data with expanded_fire_data_part3_normal (lon, lat, DATE)
# print the # of rows in weather_data before removing (in sentence)
print(f'weather_data has {weather_data.shape[0]} rows before merging')
# merge expanded_fire_data_part3_normal with weather_data
weather_data = weather_data.merge(expanded_fire_data, on=['lon', 'lat', 'day'], how='left')
# print the # of rows in weather_data after merging (in sentence)
print(f'weather_data has {weather_data.shape[0]} rows after merging')

weather_data has 147252000 rows before merging
weather_data has 147252000 rows after merging


In [200]:
# remove rows when IS_DISCOVERY_DATE is False
# print the # of rows in weather_data before removing (in sentence)
print(f'weather_data has {weather_data.shape[0]} rows before removing IS_DISCOVERY_DATE is False')
# only keep rows when IS_DISCOVERY_DATE is True or NA
weather_data = weather_data[weather_data['IS_DISCOVERY_DATE'].isna() | weather_data['IS_DISCOVERY_DATE']]
# print the # of rows in weather_data after removing (in sentence)
print(f'weather_data has {weather_data.shape[0]} rows after removing IS_DISCOVERY_DATE is False')

weather_data has 147252000 rows before removing IS_DISCOVERY_DATE is False
weather_data has 145766023 rows after removing IS_DISCOVERY_DATE is False


In [201]:
# check IS_ALARM_DATE freq table
weather_data['IS_DISCOVERY_DATE'].value_counts()

IS_DISCOVERY_DATE
True    151834
Name: count, dtype: int64

In [202]:
# if IS_ALARM_DATE is NA, fill it with 0, else fill it with 1
weather_data['IS_DISCOVERY_DATE'] = weather_data['IS_DISCOVERY_DATE'].fillna(0).astype(int)

In [203]:
# rename IS_ALARM_DATE to IS_FIRE
weather_data = weather_data.rename(columns={'IS_DISCOVERY_DATE': 'IS_FIRE'})

In [204]:
weather_data.shape

(145766023, 5)

In [205]:
weather_data = weather_data[weather_data['day'] <= '2020-12-31']

In [206]:
weather_data.shape

(126571680, 5)

In [207]:
weather_data

,lon,lat,day,IS_FIRE,NWCG_CAUSE_CLASSIFICATION
0,-124.391667,40.441667,2001-01-01,0,NaN
1,-124.391667,40.441667,2001-01-02,0,NaN
2,-124.391667,40.441667,2001-01-03,0,NaN
3,-124.391667,40.441667,2001-01-04,0,NaN
4,-124.391667,40.441667,2001-01-05,0,NaN
...,...,...,...,...,...
147250900,-115.933333,32.650000,2020-12-27,0,NaN
147250901,-115.933333,32.650000,2020-12-28,0,NaN
147250902,-115.933333,32.650000,2020-12-29,0,NaN
147250903,-115.933333,32.650000,2020-12-30,0,NaN


In [208]:
# check missing rate of NWCG_CAUSE_CLASSIFICATION
weather_data['NWCG_CAUSE_CLASSIFICATION'].isnull().mean()

0.998800410960809

In [210]:
# check mean of IS_FIRE
1-weather_data['IS_FIRE'].mean()

0.998800410960809

In [211]:
# write weather_data
weather_data.to_parquet('../Clean_Data/Fire_Data/calfire_fod_fpa_2001_2020_fire_label.parquet')

## Add Fire Size to label

In [212]:
# read fire_data
fire_label = pd.read_parquet('../Clean_Data/Fire_Data/calfire_fod_fpa_2001_2020_fire_label.parquet')

In [213]:
fire_label['IS_FIRE'].value_counts()

IS_FIRE
0    126419846
1       151834
Name: count, dtype: int64

In [214]:
fire_data = pd.read_parquet('../Clean_Data/Fire_Data/calfire_fod_fpa_2001_2020_dedup_map_weather_grid.parquet')

In [215]:
fire_data.shape

(175257, 12)

In [216]:
# group by ['NEAREST_LON','NEAREST_LAT','DISCOVERY_DATE','CONT_DATE'], and get min of column 'min_FIRE_SIZE', max of column 'max_FIRE_SIZE'
fire_data = fire_data.groupby(['NEAREST_LON','NEAREST_LAT','DISCOVERY_DATE']).agg({'min_FIRE_SIZE': 'min', 'max_FIRE_SIZE': 'max'}).reset_index()

In [217]:
# rename column to lon, lat, day
fire_data = fire_data.rename(columns={'NEAREST_LON': 'lon', 'NEAREST_LAT': 'lat', 'DISCOVERY_DATE': 'day'})

In [218]:
# assign datetime to day
fire_data['day'] = pd.to_datetime(fire_data['day'])

In [219]:
fire_data.shape

(169151, 5)

In [61]:
fire_data.head()

,lon,lat,day,min_FIRE_SIZE,max_FIRE_SIZE
0,-124.391667,40.400000,2007-07-14,0.1,0.1
1,-124.391667,40.441667,2012-10-20,0.1,0.1
2,-124.391667,40.441667,2001-07-17,0.1,0.1
3,-124.391667,40.441667,2014-07-30,100.0,100.0
4,-124.391667,40.441667,2005-09-24,0.1,0.1


In [220]:
fire_label = fire_label.merge(fire_data, on=['lon', 'lat', 'day'], how='left')

In [63]:
fire_label.shape

(126571680, 6)

In [64]:
fire_label.head()

,lon,lat,day,IS_FIRE,min_FIRE_SIZE,max_FIRE_SIZE
0,-124.391667,40.441667,2001-01-01,0,NaN,NaN
1,-124.391667,40.441667,2001-01-02,0,NaN,NaN
2,-124.391667,40.441667,2001-01-03,0,NaN,NaN
3,-124.391667,40.441667,2001-01-04,0,NaN,NaN
4,-124.391667,40.441667,2001-01-05,0,NaN,NaN


In [221]:
# fill NA with 0 for column min_FIRE_SIZE and max_FIRE_SIZE
fire_label['min_FIRE_SIZE'] = fire_label['min_FIRE_SIZE'].fillna(0)
fire_label['max_FIRE_SIZE'] = fire_label['max_FIRE_SIZE'].fillna(0)

In [222]:
# check the sum of min_FIRE_SIZE and max_FIRE_SIZE grouped by IS_FIRE
fire_label.groupby('IS_FIRE').agg({'min_FIRE_SIZE': 'sum', 'max_FIRE_SIZE': 'sum'})

,min_FIRE_SIZE,max_FIRE_SIZE
IS_FIRE,,
0,0.000000e+00,0.000000e+00
1,1.443410e+07,1.575404e+07


In [223]:
# write fire_label
fire_label.to_parquet('../Clean_Data/Fire_Data/calfire_fod_fpa_2001_2020_fire_label_w_size.parquet')